In [134]:
from pathlib import Path
import requests
import html
from time import sleep
from typing import Tuple
from pprint import pprint
from datetime import datetime

from bs4 import BeautifulSoup
import pandas as pd

In [135]:
article_list = {}

In [136]:
URL = "https://www.aitimes.com/news/articleList.html?view_type=sm"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/"
}

response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
sections = soup.select_one('#section-list')
articles = sections.select('ul > li .altlist-webzine-content')

for article in articles:
    href = article.find('a').get('href', '')
    title = article.find('h2', class_='altlist-subject').get_text(strip=True)
    summary = article.find('p', class_='altlist-summary').get_text(strip=True)
    infos = article.find_all('div', class_='altlist-info-item')
    tag, author, date = [info.get_text(strip=True) for info in infos]
    print(f"{title}\n {summary}\n {tag} | {author} | {date}\n {href}\n")
    article_list[title] = {
        'summary': summary,
        'tag': tag,
        'author': author,
        'date': date,
        'href': href
    }


아마존, AI 확대로 1만4000명 해고 발표
 아마존이 알려진 대로 본격적인 감원 절차에 돌입했다. 우선 전 세계 직원 1만4000명을 해고한다고 밝혔으며, 앞으로 더 많은 감축을 예고했다.로이터는 28일(현지시간) 내부 관계자를 인용, 아마존이 이날 오전부터 1만4000명에게 해고 통보를 전했다고 보도했다. 기기와 광고, 프라인 비디오, HR, 운영, 알렉사 담당과 AWS 등 거의 전 부서가 포함됐다.베스 갈레티 인사 및 기술 담당 부사장은 해고자들에게 보낸 메일을 통해 "더 이상 아마존을 대신해 업무를 수행할 필요가 없다"라며 인사부 직원과 화상으로 만날 수 있다고 적혀있다
 산업일반 | 임대준 기자 | 10-29 14:43
 https://www.aitimes.com/news/articleView.html?idxno=203548

구글 클라우드, ‘비오 3’ 국내 엔터 산업에 적용...”K팝 뮤비도 AI로 생성”
 구글 클라우드가 동영상 생성 인공지능(AI) 모델 ‘비오 3(Veo 3)’를 도입한 국내 기업들의 성공 사례를 29일 공개했다.지기성 구글 클라우드 코리아 사장은 “비오 3는 콘텐츠 제작 전반에서 영상 제작의 장벽을 혁신적으로 낮추며, 전문적인 수준의 영상을 빠르게 제작할 수 있는 강력한 도구”라며 “비오 3를 통해 국내 미디어 및 엔터테인먼트 기업들이 차세대 콘텐츠 혁신을 주도하고 크리에이티브 경쟁력을 강화할 수 있도록 적극적으로 협력하고자 한다”라고 말했다.필굿뮤직을 첫번째 성공 사례로 들었다. 타이거 JK가 이끄는 국내 대표
 AI 기업 | 장세민 기자 | 10-29 14:07
 https://www.aitimes.com/news/articleView.html?idxno=203543

최수연 네이버 대표, APEC CEO 서밋 기조연설...”AI 데이터센터 지원책 마련 강조”
 최수연 네이버 대표는 29일 경북 경주 예술의전당에서 열린 APEC CEO 서밋에 참석해 네이버의 풀스택 인공지능(AI) 구축 및 운영 경험과 혁신 방향을 소개,

In [137]:
def decode_html_entities(obj):
    if isinstance(obj, str):
        return html.unescape(obj)
    elif isinstance(obj, dict):
        return {k: decode_html_entities(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [decode_html_entities(item) for item in obj]
    else:
        return obj

In [138]:
def article_content(title: str, href: str) -> Tuple[list, str]:
    try:
        response = requests.get(href, headers=headers, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        article_section = soup.select_one('#article-view-content-div')
        imgs = article_section.select('img')
        paragraphs = article_section.select('p')
        paragraphs.pop()
        img_urls = [img.get('src', '') for img in imgs]
        text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        # print(text)
        sleep(1)
        return img_urls, text
    except requests.exceptions.RequestException as e:
        print(f'"{title}"의 기사를 읽던 중 에러가 발생했습니다.')
        raise

In [ ]:
def get_article_info(page: int) -> dict:
    base_url = 'https://www.aitimes.com/news/ajaxArticlePaging.php'
    base_href = 'https://www.aitimes.com'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.aitimes.com/news/articleList.html?view_type=sm',
    }

    params = {
        # 'total': 28636,
        # 'list_per_page': 20,
        # 'page_per_page': 10,
        'page': page,
        'view_type': 'sm',
        # 'box_idxno': 0,
    }

    response = requests.get(base_url, headers=headers, params=params, timeout=5)
    articles = {}
    try:
        results = response.json()['data']
        decoded_results = decode_html_entities(results)
        
        for item in decoded_results:
            title = item['title']
            author = item['user_name']
            date = item['approve_date']
            href = base_href + item['link_url']
            tag = item['sectionTitle']
            summary = item['summaryNoimg']
            # print("="*50)
            # print(f"{title}\n{summary}\n{tag} | {author} | {date}\n {href}")
            # print('='*50)
            img_urls, text = article_content(title, href)
            articles[title] = {
                'summary': summary,
                'tag': tag,
                'author': author,
                'date': date
                'href': href,
                'content': {
                    'img_urls': img_urls,
                    'text': text
                }
            }
        return articles
    except requests.exceptions.RequestException as e:
        print(f'{page} 페이지 요청 중 에러 발생! {e}')

SyntaxError: invalid syntax (1592740487.py, line 42)

: 

In [ ]:
articles = {}


for page in range(1, 6):
    page_articles = get_article_info(page)
    articles.update(page_articles)
    sleep(1)

In [ ]:
articles

In [ ]:
# 이런 비정형 데이터는 JSON 형태로 저장하고, MongoDB 같은 NoSQL 데이터베이스로 관리하는 것이 좋습니다.